In [1]:
import pandas as pd
import json
import requests
from config import api_key
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from flask import Flask, jsonify, render_template

In [2]:
def get_businesses(location, term, api_key):
    headers = {'Authorization': 'Bearer %s' % api_key}
    url = 'https://api.yelp.com/v3/businesses/search'

    data = []
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': location.replace(' ', '+'),
            'term': term.replace(' ', '+'),
            'offset': offset
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break

    return data

In [12]:
#location can be entered using city name or zipcode
location = "San Francisco"
term = "restaurant"
response = get_businesses(location, term, api_key)

In [13]:
business_name = []
business_review_count = []
avg_rating = []
lat = []
long = []
address = []
phone = []

for result in response:
    business_name.append(result["name"])
    business_review_count.append(result["review_count"])
    avg_rating.append(result["rating"])
    lat.append(result["coordinates"]["latitude"])
    long.append(result["coordinates"]["longitude"])
    address.append(result["location"]["display_address"][0])
    phone.append(result["display_phone"])

In [14]:
result_df = pd.DataFrame({ "Business_Name" : business_name,
                           "Review_Count" : business_review_count,
                           "Average_Rating" : avg_rating,
                           "Latitude" : lat,
                           "Longtitude" : long,
                           "Address" : address,
                           "Phone_Number" : phone })
result_df = result_df.astype({"Business_Name": str, "Review_Count": int, "Average_Rating": float,
                              "Latitude": float, "Longtitude": float, "Address":str, "Phone_Number": str})
result_df

,Business_Name,Review_Count,Average_Rating,Latitude,Longtitude,Address,Phone_Number
0,El Fuego,19,5.0,37.764910,-122.400650,17TH St And Carolina St,(415) 465-9395
1,Fog Harbor Fish House,6071,4.0,37.809041,-122.410233,The Embarcadero,(415) 421-2442
2,Chicken as Cluck,104,4.0,37.750586,-122.395178,1760 Cesar Chavez St,(415) 670-0428
3,Marufuku Ramen SF,2990,4.5,37.785014,-122.431842,1581 Webster St,(415) 872-9786
4,Dumpling Time,1857,4.0,37.769753,-122.402255,11 Division St,(415) 525-4797
...,...,...,...,...,...,...,...
995,Hang Ah Tea Room,775,3.5,37.793350,-122.407490,1 Pagoda Pl,(415) 982-5686
996,Mizutani Sushi Bar,188,4.0,37.785198,-122.464859,4406 California St,(415) 668-3288
997,Park Tavern,1575,4.0,37.801130,-122.409000,1652 Stockton St,(415) 989-7300
998,Street Taco,35,4.0,37.766471,-122.396742,980 16th St,


In [17]:
engine = create_engine("sqlite:///sf_restaurant_db.sqlite")
conn = engine.connect()
Base = declarative_base()

In [18]:
class Restaurant(Base):
    __tablename__ = "restaurant_info"
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key = True)
    Business_Name = Column(String(50))
    Review_Count = Column(Integer)
    Average_Rating = Column(Float)
    Latitude = Column(Float)
    Longtitude = Column(Float)
    Address = Column(String(200))
    Phone_Number = Column(String(30))

In [19]:
Base.metadata.create_all(conn)

In [20]:
result_df.to_sql("restaurant_info", con=engine, if_exists='replace', index_label='id')

In [45]:
restaurant_data = pd.read_sql_query('select * from restaurant_info', con=engine)
restaurant_data

,id,Business_Name,Review_Count,Average_Rating,Latitude,Longtitude,Address,Phone_Number
0,0,El Fuego,19,5.0,37.764910,-122.400650,17TH St And Carolina St,(415) 465-9395
1,1,Fog Harbor Fish House,6071,4.0,37.809041,-122.410233,The Embarcadero,(415) 421-2442
2,2,Chicken as Cluck,104,4.0,37.750586,-122.395178,1760 Cesar Chavez St,(415) 670-0428
3,3,Marufuku Ramen SF,2990,4.5,37.785014,-122.431842,1581 Webster St,(415) 872-9786
4,4,Dumpling Time,1857,4.0,37.769753,-122.402255,11 Division St,(415) 525-4797
...,...,...,...,...,...,...,...,...
995,995,Hang Ah Tea Room,775,3.5,37.793350,-122.407490,1 Pagoda Pl,(415) 982-5686
996,996,Mizutani Sushi Bar,188,4.0,37.785198,-122.464859,4406 California St,(415) 668-3288
997,997,Park Tavern,1575,4.0,37.801130,-122.409000,1652 Stockton St,(415) 989-7300
998,998,Street Taco,35,4.0,37.766471,-122.396742,980 16th St,


In [59]:
session = Session(bind=engine) 
best_restaurant = session.query(Restaurant).filter(Restaurant.Average_Rating >= 4).filter(Restaurant.Review_Count > 2000).all()

In [61]:
for item in best_restaurant:
    print(item.Business_Name)

Fog Harbor Fish House
Marufuku Ramen SF
San Tung
Burma Superstar
Sotto Mare Oysteria & Seafood
Hunan Home's Restaurant
The Codmother Fish & Chips
The Front Porch
The House
Farmhouse Kitchen Thai Cuisine
Limón Rotisserie
Kokkari Estiatorio
Cha Cha Cha
Kin Khao
Wayfare Tavern
Chez Maman East
Suppenküche
Hops & Hominy
El Farolito
Pearl's Deluxe Burgers
The Monk's Kettle
La Taqueria
Golden Boy Pizza
Mensho Tokyo
Gary Danko
Woodhouse Fish Company
House of Prime Rib
Hard Knox Cafe
Lers Ros
Swan Oyster Depot
Mandalay
Pancho Villa Taqueria
Hog Island Oyster Co
Little Star Pizza
Thanh Long
Nopa
Tommy's Joynt
Zuni Café
Delfina
Izakaya Sozai
Coqueta
Greens Restaurant
Marlowe
Tony's Pizza Napoletana
Nick's Crispy Tacos
Gracias Madre
La Mar Cebichería Peruana
Scoma's Restaurant
Amber India
Boudin Bakery & Cafe
Loló
Perbacco
Tadich Grill
DOSA on Fillmore
Pizzeria Delfina - Mission
Chapeau!
Little Star Pizza
PPQ Dungeness Island
HRD
Brenda's French Soul Food
È Tutto Qua
Bobo's
Liholiho Yacht Club
Bou